In [ ]:
Language = "English"
# Language = "Chinese"
mode = "train"
param_num = 1

In [ ]:
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

from Part1.dataprocess import data_process, set_log, combine_data
from sklearn_crf import sent2features


set_log(None)
train_data, valid_data, test_data = data_process(f"../NER/{Language}", mode=mode)

x_train = [sent2features(sentence, Language, param_num) for sentence, _ in train_data]
y_train = [label for _, label in train_data]
x_valid = [sent2features(sentence, Language, param_num) for sentence, _ in valid_data]
y_valid = [label for _, label in valid_data]

In [ ]:
import pickle
from sklearn_crfsuite import CRF

if param_num == 0:
    crf_model = CRF(
        algorithm="lbfgs",
        c1=0.01,
        c2=0.01,
        max_iterations=200,
        all_possible_transitions=True,
        verbose=True,
    )
elif param_num == 1:
    crf_model = CRF(
        algorithm="ap",
        max_iterations=300,
        all_possible_transitions=True,
        verbose=True,
    )
elif param_num == 2:
    raise NotImplementedError

crf_model.fit(x_train, y_train)
with open(f"crf_{Language}{param_num}.pkl", "wb") as f:
    pickle.dump(crf_model, f)

y_pred = crf_model.predict(x_valid)
combined_data = combine_data([sentence for sentence, _ in valid_data], y_pred)

output_file = f"output_{Language}.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(combined_data)

In [ ]:
from NER.check import check

report = check(
    language=Language,
    gold_path=f"../NER/{Language}/validation.txt",
    my_path=output_file,
)